In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from pmdarima import auto_arima

# 读取数据
data = pd.read_csv("sorted_final_transactions_dataset.csv")
data['date_BUY_fix'] = pd.to_datetime(data['date_BUY_fix'])
data.sort_values(by='date_BUY_fix', inplace=True)

# 按公司分组
grouped_data = data.groupby('company')

# 预测时间点
forecast_periods = [90, 180, 270, 365]  # 3个月，6个月，9个月，12个月

In [2]:
inflation_rates = {
        2013: 1.41,
        2014: -0.15,
        2015: -0.5,
        2016: -0.2,
        2017: 0.96,
        2018: 0.68
}

In [3]:
def investment_evaluation(company, buy_date, holding_time, data):
    buy_date = pd.to_datetime(buy_date)
    sell_date = buy_date + pd.DateOffset(days=holding_time)
    
    company_data = data[(data['company'] == company) & (data['date_BUY_fix'] < buy_date)]
    company_data = company_data.drop_duplicates(subset=['date_BUY_fix'])
    company_data.sort_values(by='date_BUY_fix', inplace=True)

    model = auto_arima(company_data['price_BUY'], seasonal=False, trace=True)
    fitted_model = model.fit(company_data['price_BUY'])
    forecast = fitted_model.predict(n_periods=holding_time).iloc[-1]
    buy_price = company_data.iloc[-1]['price_BUY']
    nominal_return = (forecast - buy_price) / buy_price * 100

    # Calculate real return adjusting for inflation
    buy_year = buy_date.year
    sell_year = sell_date.year
    
    # Calculate average inflation from buy year to sell year
    if buy_year != sell_year:
        avg_inflation = np.mean([inflation_rates[year] for year in range(buy_year, sell_year + 1) if year in inflation_rates])
    else:
        avg_inflation = inflation_rates.get(buy_year, 0)  # Default to 0 inflation if no data available
    
    real_return = nominal_return - avg_inflation

    # Determine if the investment is good
    is_good = "Good" if nominal_return > 0 else "Bad"

    return {
        "Company": company,
        "Buy Date": buy_date.strftime('%Y-%m-%d'),
        "Forecasted Price": forecast,
        "Nominal Return (%)": nominal_return,
        "Inflation Adjusted Return (%)": real_return,
        "Investment Evaluation": is_good
    }



In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Function to convert series to supervised learning format
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # Forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

for company, company_data in grouped_data:
    company_data = company_data.drop_duplicates(subset=['date_BUY_fix'])

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(company_data[['price_BUY']])

    reframed = series_to_supervised(scaled, 1, 1)

    # Split into train and test sets based on the cutoff date
    train_data = company_data[company_data['date_BUY_fix'] < '2017-01-01']
    test_data = company_data[company_data['date_BUY_fix'] >= '2017-01-01']

    if len(train_data) == 0 or len(test_data) == 0:
        print(f"Skipping company {company} due to insufficient data.")
        continue

    train_scaled = scaler.transform(train_data[['price_BUY']])
    test_scaled = scaler.transform(test_data[['price_BUY']])
    train_reframed = series_to_supervised(train_scaled, 1, 1)
    test_reframed = series_to_supervised(test_scaled, 1, 1)

    train = train_reframed.values
    test = test_reframed.values
    train_X, train_y = train[:, :-1], train[:, -1]
    test_X, test_y = test[:, :-1], test[:, -1]

    # Check if there is enough data after reframing
    if len(train_X) == 0 or len(test_X) == 0:
        print(f"Skipping company {company} due to insufficient data after reframing.")
        continue

    train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
    test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

    # Design LSTM network
    model = Sequential()
    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')

    # Fit network
    history = model.fit(train_X, train_y, epochs=20, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)

    yhat = model.predict(test_X)
    test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
    inv_yhat = np.concatenate((yhat, test_X[:, 1:]), axis=1)
    inv_yhat = scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]

    from sklearn.metrics import mean_squared_error
    rmse = np.sqrt(mean_squared_error(test_y, yhat))
    print(f'RMSE for company {company}: {rmse:.2f}')

    actual_prices = test_data['price_BUY']

    test_dates_adjusted = test_data['date_BUY_fix'].iloc[1:].reset_index(drop=True)  # Drop the first date to align with predictions
    actual_prices = test_data['price_BUY'].iloc[1:].reset_index(drop=True)  # Drop the first date to align with predictions

    # plt.figure(figsize=(10, 6))
    plt.plot(train_data['date_BUY_fix'], train_data['price_BUY'], label='Train data')
    plt.plot(test_dates_adjusted, actual_prices, label='Actual prices')
    plt.plot(test_dates_adjusted, inv_yhat, label='Predicted prices')
    plt.title(f"Stock Prices Prediction for Company {company}")
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

In [24]:
result = investment_evaluation(17, "2015-5-22", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.45 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=756.222, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=758.148, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=758.133, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=756.651, Time=0.02 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.37 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.963 seconds
{'Company': 17, 'Buy Date': '2015-05-22', 'Forecasted Price': 52.27695344187996, 'Nominal Return (%)': 2.84373124999517, 'Inflation Adjusted Return (%)': 3.34373124999517, 'Investment Evaluation': 'Good'}


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [26]:
result = investment_evaluation(21, "2014-8-22", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=367.846, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=371.452, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=370.314, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=370.129, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=371.124, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=367.178, Time=0.09 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=372.017, Time=0.05 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=371.443, Time=0.11 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=368.219, Time=0.12 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=373.837, Time=0.07 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=366.335, Time=0.13 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=371.942, Time=0.08 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=367.786, Time=0.11 sec
 ARIMA(3,1,0)(0,0,0)[0] intercept   : AIC=372.860, Time=0.05 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=369.959, Time=0.15 sec
 ARIMA(2,1,1)(0,0,0)[0]             : AIC=366.652, Time=0.08 sec

Best model:  ARIMA(2,1,1)(0,0,0)[0] intercept
Total fit time: 1.384 seconds
{'Company': 21, 'Buy Date': '2014-08-22', 'Forecasted Price': 77.65613298196001, 'Nominal Return (%)': 1.9752429462702645, 'Inflation Adjusted Return (%)': 2.1252429462702644, 'Investment Evaluation': 'Good'}


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [27]:
result = investment_evaluation(13, "2017-5-30", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2267.345, Time=0.49 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=2267.120, Time=0.08 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=2268.263, Time=0.05 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=2268.161, Time=0.08 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=2265.183, Time=0.02 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2264.681, Time=0.32 sec


...

/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2262.784, Time=0.10 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=2266.229, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=2266.331, Time=0.03 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1


 ARIMA(2,1,1)(0,0,0)[0]             : AIC=2263.652, Time=0.14 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=2263.701, Time=0.15 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=2265.015, Time=0.06 sec
 ARIMA(2,1,0)(0,0,0)[0]             : AIC=2265.796, Time=0.05 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=2265.448, Time=0.25 sec

Best model:  ARIMA(1,1,1)(0,0,0)[0]          
Total fit time: 2.666 seconds
{'Company': 13, 'Buy Date': '2017-05-30', 'Forecasted Price': 47.08956077412071, 'Nominal Return (%)': 0.14285990520261807, 'Inflation Adjusted Return (%)': -0.8171400947973819, 'Investment Evaluation': 'Good'}


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [28]:
result = investment_evaluation(21, "2017-1-10", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.62 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1749.337, Time=0.06 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1751.231, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1751.220, Time=0.07 sec


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1749.060, Time=0.03 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.84 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 1.673 seconds
{'Company': 21, 'Buy Date': '2017-01-10', 'Forecasted Price': 91.92563629, 'Nominal Return (%)': 0.0, 'Inflation Adjusted Return (%)': -0.96, 'Investment Evaluation': 'Bad'}


/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: invalid value encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/Users/liuyichen/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [38]:
result = investment_evaluation(2, "2018-8-31", 78, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=3212.947, Time=0.31 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3215.984, Time=0.09 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3217.265, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3217.119, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3214.987, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=3212.443, Time=0.20 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3210.815, Time=0.10 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=3212.109, Time=0.12 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3215.265, Time=0.19 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=3213.106, Time=0.34 sec
 ARIMA(0,1,2)(0,0,0)[0]             : AIC=3209.928, Time=0.06 sec
 ARIMA(0,1,1)(0,0,0)[0]             : AIC=3216.065, Time=0.03 sec
 ARIMA(1,1,2)(0,0,0)[0]             : AIC=3211.538, Time=0.12 sec
 ARIMA(0,1,3)(0,0,0)[0]             : AIC=3211.177, Time=0.07 sec
 ARIMA(1,1,1)(0,0,0)[0]          

In [40]:
result = investment_evaluation(7, "2017-5-1", 90, data)
print(result)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=1401.135, Time=0.54 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=1406.631, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=1408.426, Time=0.05 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=1408.444, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=1406.977, Time=0.02 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=1410.201, Time=0.18 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=1410.363, Time=0.19 sec
 ARIMA(3,1,2)(0,0,0)[0] intercept   : AIC=1408.994, Time=0.73 sec
 ARIMA(2,1,3)(0,0,0)[0] intercept   : AIC=1409.102, Time=0.75 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=1409.962, Time=0.17 sec
 ARIMA(1,1,3)(0,0,0)[0] intercept   : AIC=1409.371, Time=0.64 sec
 ARIMA(3,1,1)(0,0,0)[0] intercept   : AIC=1409.416, Time=0.66 sec
 ARIMA(3,1,3)(0,0,0)[0] intercept   : AIC=1404.597, Time=0.86 sec
 ARIMA(2,1,2)(0,0,0)[0]             : AIC=1401.358, Time=0.39 sec

Best model:  ARIMA(2,1,2)(0,0,0)

In [41]:
result = investment_evaluation(18, "2014-9-24", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.34 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=247.586, Time=0.02 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=249.301, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=249.288, Time=0.04 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=249.350, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=inf, Time=0.15 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0] intercept
Total fit time: 0.603 seconds
{'Company': 18, 'Buy Date': '2014-09-24', 'Forecasted Price': 42.63775294744418, 'Nominal Return (%)': 3.735872304655488, 'Inflation Adjusted Return (%)': 3.885872304655488, 'Investment Evaluation': 'Good'}


In [42]:
result = investment_evaluation(27, "2015-6-29", 30, data)
print(result)

Performing stepwise search to minimize aic
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=inf, Time=0.41 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=801.393, Time=0.03 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=801.243, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=801.348, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=799.394, Time=0.01 sec
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=803.100, Time=0.08 sec

Best model:  ARIMA(0,1,0)(0,0,0)[0]          
Total fit time: 0.591 seconds
{'Company': 27, 'Buy Date': '2015-06-29', 'Forecasted Price': 63.36894989, 'Nominal Return (%)': 0.0, 'Inflation Adjusted Return (%)': 0.5, 'Investment Evaluation': 'Bad'}
